Import Packages

In [6]:
# !pip install folium
# !pip install pulp
# !pip install openpyxl
# !pip install geopandas
# !pip install kaleido
# !python -m pip install global_land_mask
import kaleido
kaleido.__version__ #0.2.1
import pulp #conda install -c conda-forge pulp
import numpy as np
import pandas as pd #conda install pandas
import math
import json
import geopandas as gpd
import folium
import plotly.express as px
import plotly.graph_objects as go
import requests
from global_land_mask import globe
import time
import constants # constants.py

In [7]:
# Generate the grid coordinates
grid = []
for lat in range(int(constants.min_lat), int(constants.max_lat) + 1):
    for lon in range(int(constants.min_lon), int(constants.max_lon) + 1):
        if globe.is_land(lat, lon):  # check if coordinates are on land
            for i in range(int(constants.grid_spacing)):
                for j in range(int(constants.grid_spacing)):
                    point_lat = lat + i / constants.grid_spacing
                    point_lon = lon + j / constants.grid_spacing
                    params = { # API to check the country the coordinates are in
                        'key': constants.api_key_opencage,
                        'q': f"{point_lat},{point_lon}",
                        'no_annotations': 1
                    }
                    response = requests.get(constants.base_url_opencage, params=params).json()
                    if 'results' in response and len(response['results']) > 0:
                        country = response['results'][0]['components']['country']
                        grid.append((point_lat, point_lon, country))

# Create a DataFrame with the grid coordinates and country information
df = pd.DataFrame(grid, columns=['Latitude', 'Longitude', 'Country'])

# Reorder the columns
df = df[['Country', 'Latitude', 'Longitude']]

# Add an index column for the Excel file
df = df.reset_index()

# Rename the 'index' column
df = df.rename(columns={'index': 'Name'})

# Replace name of countries so they match the Fixed Cost df
df['Country'] = df['Country'].replace('Czechia','Czech Republic')

# Add a prefix "N" to the Facility name number
df['Name'] = 'N' + df['Name'].astype(str)

# Save the DataFrame to an Excel file
df.to_excel('Facilities.xlsx', index=True)